# LSH Implementation 

The Implementation of the project is divided into four substeps which includes::

## 1. **Reading the Input Content and Text Preprocessing**

This involves loading data from the following files:
- `ids.txt`: Contains the document IDs.
- `texts.txt`: Contains the text data corresponding to the document IDs.

Text preprocessing typically involves several operations:
- **Lowercase conversion**: Ensures uniformity by converting all text to lowercase.
- **Removing punctuation**: Cleans the text to focus on words.
- **Tokenization**: Splits the text into individual words or tokens.

Further, the preprocessing may also include:
- **Stopword removal**: Eliminates common words (e.g., "the", "is", "and") which do not contribute much meaning.
- **Stemming/Lemmatization**: Reduces words to their base forms (e.g., "running" → "run") to standardize the text, improving the comparison process.

## 2. **Vectorizing the Text with TF-IDF**

**TF-IDF** (Term Frequency-Inverse Document Frequency) is a technique used to convert text into numerical vectors. The process weighs terms based on two factors:

- **Term Frequency (TF)**: Measures how often a word appears in a document.
- **Inverse Document Frequency (IDF)**: Measures how rare a word is across all documents.

This method captures the relative importance of words, allowing for meaningful comparisons between documents. TF-IDF representation is effective for identifying key terms and measuring document similarity.

## 2. **Vectorizing the Text with TF-IDF**

**TF-IDF** (Term Frequency-Inverse Document Frequency) is a technique used to convert text into numerical vectors. The process weighs terms based on two factors:

- **Term Frequency (TF)**: Measures how often a word appears in a document.
- **Inverse Document Frequency (IDF)**: Measures how rare a word is across all documents.

This method captures the relative importance of words, allowing for meaningful comparisons between documents. TF-IDF representation is effective for identifying key terms and measuring document similarity.

---

### Proper Working of TF-IDF:

**TF-IDF** stands for Term Frequency-Inverse Document Frequency. It measures how important a word is in a document within a collection of documents.

The TF-IDF value for a term is calculated by multiplying two metrics:

1. **Term Frequency (TF)**: The number of times a term appears in a document, normalized by the total number of terms in the document.

   $$
   TF(t, d) = \frac{\text{Number of times term } t \text{ appears in document } d}{\text{Total number of terms in document } d}
   $$

2. **Inverse Document Frequency (IDF)**: A measure of how rare a term is across all documents. The more rare a term is, the higher its IDF value.

   $$
   IDF(t) = \log \left(\frac{N}{n_t}\right)
   $$

   Where:
   - \( N \) = Total number of documents
   - \( n<sub>t</sub> \) = Number of documents containing the term \( t \)

The final **TF-IDF** score for a term is the product of the term frequency and inverse document frequency:

$$
\text{TF-IDF}(t, d) = TF(t, d) \times IDF(t)
$$
---

By applying this formula, TF-IDF identifies important words in the document that are also relatively unique across the corpus.


## 3. **Implementing LSH for Finding Similarity**

**LSH** (Locality-Sensitive Hashing) is used to approximate similarity search, enabling efficient identification of similar documents. It works by:
- Hashing similar items into the same "buckets" with high probability.

For text data, **MinHash** is commonly employed. It creates multiple hash functions to generate a signature for each document, allowing quick retrieval of potential similar items without performing exhaustive pairwise comparisons.

---

## 4. **Evaluating the Accuracy by Comparing with Ground Truth**

This step evaluates the performance of the LSH implementation by comparing its results to the provided ground truth in the `items.json` file. The evaluation process involves:
- Calculating **intersection scores** between the predicted and actual similar items for each document.
- Computing the **average score** across all documents to assess the overall model performance.



In [14]:
# Installing the required libraries which might not be available in the environment.
!pip install scikit-learn numpy pandas matplotlib
!pip install numpy pandas matplotlib scikit-learn
!pip install numpy matplotlib seaborn pandas datasketch
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [15]:
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datasketch import MinHash, MinHashLSH
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import re
from nltk.util import ngrams

In [16]:
def create_ngrams(words, n=2):
    return [' '.join(gram) for gram in ngrams(words, n)]


def create_shingles(text, n=2):
    words = preprocess_text(text)
    unigrams = words
    bigrams = create_ngrams(words, 2)
    trigrams = create_ngrams(words, 3)
    return set(unigrams + bigrams + trigrams)


def create_minhash(shingles, num_perm=256):
    minhash = MinHash(num_perm=num_perm)
    for shingle in shingles:
        minhash.update(shingle.encode('utf-8'))
    return minhash


def build_lsh_index(ids, texts, threshold=0.5, num_perm=256):
    lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
    for id, text in zip(ids, texts):
        shingles = create_shingles(text)
        lsh.insert(id, create_minhash(shingles, num_perm))
    return lsh


In [17]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0


def find_similar_items(lsh, ids, texts, num_results=5):
    similar_items = {}
    for id, text in zip(ids, texts):
        query_shingles = create_shingles(text)
        query_minhash = create_minhash(query_shingles)
        results = lsh.query(query_minhash)
        results = [r for r in results if r != id]

        similarities = []
        for result_id in results:
            result_shingles = create_shingles(texts[ids.index(result_id)])
            similarity = jaccard_similarity(query_shingles, result_shingles)
            similarities.append((result_id, similarity))

        similarities.sort(key=lambda x: x[1], reverse=True)
        similar_items[id] = [item[0] for item in similarities[:num_results]]

    return similar_items


def evaluate_model(similar_items, ground_truth):
    intersection_scores = []
    for id, predicted in similar_items.items():
        truth = ground_truth.get(id, [])
        intersection = set(predicted) & set(truth)
        intersection_scores.append(len(intersection))
    return intersection_scores


In [18]:
def read_input_files(ids_file, texts_file, items_file):
    with open(ids_file, 'r', encoding='utf-8') as f:
        ids = [line.strip() for line in f]
    with open(texts_file, 'r', encoding='utf-8') as f:
        texts = [line.strip() for line in f]
    with open(items_file, 'r', encoding='utf-8') as f:
        ground_truth = json.load(f)
    return ids, texts, ground_truth


def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    return text.split()

In [19]:
def visualize_results(intersection_scores):

    plt.figure(figsize=(10, 5))
    plt.hist(intersection_scores, bins=6, range=(0, 5), edgecolor='black')
    plt.title('Histogram of Intersection Scores')
    plt.xlabel('Intersection Score')
    plt.ylabel('Frequency')
    plt.savefig('histogram.png')
    plt.close()

    plt.figure(figsize=(10, 5))
    sns.boxplot(x=intersection_scores)
    plt.title('Box Plot of Intersection Scores')
    plt.xlabel('Intersection Score')
    plt.savefig('boxplot.png')
    plt.close()

    stats = pd.Series(intersection_scores).describe()
    print(stats)
    stats.to_csv('descriptive_stats.csv')

In [20]:
def main():
    ids, texts, ground_truth = read_input_files(
        'ids.txt', 'texts.txt', 'items.json')
    lsh = build_lsh_index(ids, texts)
    similar_items = find_similar_items(lsh, ids, texts)

    with open('lsh_similar_items.json', 'w', encoding='utf-8') as f:
        json.dump(similar_items, f, indent=2)
    intersection_scores = evaluate_model(similar_items, ground_truth)
    visualize_results(intersection_scores)

    average_score = sum(intersection_scores) / len(intersection_scores)
    print(f"Average Intersection Score: {average_score}")
    with open('average_score.txt', 'w', encoding='utf-8') as f:
        f.write(f"Average Intersection Score: {average_score}")


if __name__ == "__main__":
    main()

count    148928.000000
mean          0.001081
std           0.033065
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           2.000000
dtype: float64
Average Intersection Score: 0.001081059303824667
